In [17]:
import tensorflow as tf

from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, GlobalAveragePooling1D, Conv2D, ConvLSTM2D, ConvLSTM1D, Input, Flatten, Reshape
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.utils import pad_sequences
#from tensorflow.keras import ops
import keras

In [15]:
"""Vars"""
sample_headlines = ["Hoo Hoo", "HOO", "WHOSE TOES", "HOO", "Hoo hoo hoo"]
sample_prices = [34.3, 40.4, 90, 30, 0.5]
sample_gains = [0] + [(sample_prices[i+1]-sample_prices[i])/sample_prices[i] for i in range(0, len(sample_prices)-1)]
#print(sample_gains)
vocab_size = 50
max_len = 30
embeddings_dim = 1#5


In [12]:
"""Encoder Methods"""
def get_one_hot_encoded_batch(vocab_size, strings):
    return [one_hot(string, vocab_size) for string in strings]

def pad_input(max_len, one_hot_encoded_strings):
    return pad_sequences(one_hot_encoded_strings, maxlen=max_len)



In [20]:
"""Encoder Architecture"""
input = Input(shape=(max_len, vocab_size))
embeddings_1 = Embedding(input_dim=vocab_size, output_dim=embeddings_dim, input_length=max_len)(input)
#flatten_1 = Flatten()(embeddings_1)
reshape_1 = Reshape((max_len, vocab_size))(embeddings_1)
lstm_1 = LSTM(units=30, return_sequences=False)(reshape_1) # Default activation tanh - VERIFY
dense_1 = Dense(units=30)(lstm_1)
dense_2 = Dense(units=1)(dense_1)
output = dense_2 # Perhaps more to come

encoder = keras.Model(inputs = input, outputs = output)
print(encoder.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 30, 50)]          0         
                                                                 
 embedding_5 (Embedding)     (None, 30, 50, 1)         50        
                                                                 
 reshape_1 (Reshape)         (None, 30, 50)            0         
                                                                 
 lstm_4 (LSTM)               (None, 30)                9720      
                                                                 
 dense (Dense)               (None, 30)                930       
                                                                 
 dense_1 (Dense)             (None, 1)                 31        
                                                                 
Total params: 10731 (41.92 KB)
Trainable params: 10731 (41.92

In [ ]:
"""Encoder Architecture 2"""
input = Input(shape=(max_len, vocab_size))
embeddings_1 = Embedding(input_dim=vocab_size, output_dim=embeddings_dim, input_length=max_len)(input)
#flatten_1 = Flatten()(embeddings_1)
reshape_1 = Reshape((max_len, vocab_size))(embeddings_1)
lstm_1 = LSTM(units=30, return_sequences=False)(reshape_1) # Default activation tanh - VERIFY
dense_1 = Dense(units=30)(lstm_1)
dense_2 = Dense(units=1)(dense_1)
output = dense_2 # Perhaps more to come

encoder = keras.Model(inputs = input, outputs = output)
print(encoder.summary())

In [ ]:
"""Data Processing"""
